In [ ]:
# npx hardhat node --fork https://eth-mainnet.alchemyapi.io/v2/xOu9KmQYmgmqBuYhhPW0naOB9YRY3foa
# ngrok authtoken 1WA2UnsI5GHgexqADDhvw7v1Npd_dqSFiM3bX98ny1vjvx3a
# ngrok http 8545

# 初始化
require 'ethereum.rb'
require 'eth'
require 'faraday'
require 'parallel'
require 'resolv-replace'
require 'erb'

false

In [ ]:
#configure
etherscan_apikey = "545QYEP8TJKJWMZ7QP41QYXZNMJDVCJUI9"

# 参数设置
fomo_name = "FOMOFast"
fomo_addr_abi = fomo_addr = "0x324b50cAA2b95df219e78e016A3014E9B70EF5a2".downcase
fomo_addr = "0x205718799d502Fe2C45D3aFc91C3c8cCb5c0836f".downcase  #15.87 eth
fomo_addr_abi = fomo_addr = "0xC3BD383Ed5B4F682F5848E63079dF9E5b5261253".downcase #2.26 eth
fomo_addr = "0x45199Ac8507bb38A8734B104f18E881bac832c25".downcase
fomo_addr_abi = fomo_addr = "0x4e8ecF79AdE5e2C49B9e30D795517A81e0Bf00B8".downcase #86 eth
fomo_addr = "0x254176471c09b5c4f6790e000af98894459ad334".downcase
fomo_addr_abi = fomo_addr = "0x254176471c09b5c4f6790e000af98894459ad334".downcase # balance < pot


fomo_addr_abi = fomo_addr = "0xd7378939089abd6656d3f523bf55edcd2213332c".downcase #86 eth
fomo_addr = "0x05aa2fdf9f58b426b49900834cce0565d88e52eb".downcase


playerbook_name = "PlayerBook"
playerbook_addr_abi = playerbook_addr= "0x18A586F55DA3C95Fe1bD79A4E54F9a37987b688A".downcase
playerbook_addr = "0x27D5C0C175C1Ba67986319ac297d2F4D3bC2b7b2".downcase
playerbook_addr_abi = playerbook_addr = "0x27D5C0C175C1Ba67986319ac297d2F4D3bC2b7b2".downcase
#playerbook_addr = ???
playerbook_addr_abi = playerbook_addr = "0xD60d353610D9a5Ca478769D371b53CEfAA7B6E4c".downcase

# playerbook_addr = nil
# playerbook_addr_abi = playerbook_addr = nil

# 定义账户
sender_account = "0xaCC961474eb17C103BBed96FC8f0E4eEac91c475"
sender_priv_key = "6ac277496bb17f43074545db8e2e26e903aa014687448d8ed564d9881ce9bda9"
sender_key = Eth::Key.new priv: sender_priv_key

impersonate_account = "0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266"
impersonate_priv_key = "ac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
impersonate_key = Eth::Key.new priv: impersonate_priv_key


# 创建Client
$client = Ethereum::HttpClient.new("http://127.0.0.1:8545/")
$client.gas_limit = 30_000_000
$client.gas_price = $client.eth_gas_price["result"].to_i(16)

# 创建Fomo Contract
fomo_abi_url = "https://api.etherscan.io/api?module=contract&action=getabi&address=#{fomo_addr_abi}&apikey=#{etherscan_apikey}"
fomo_abi = JSON.parse(Faraday.get(fomo_abi_url).body)["result"]

$fomo_contract = Ethereum::Contract.create(
        client: $client, 
        name: fomo_name, 
        address: fomo_addr, 
        abi: fomo_abi)

$fomo_contract.key = sender_key
$fomo_contract.gas_limit = 30_000_000
$fomo_contract.gas_price = $client.eth_gas_price["result"].to_i(16)


# 创建PlayerBook Contract
playerbook_abi_url = "https://api.etherscan.io/api?module=contract&action=getabi&address=#{playerbook_addr_abi}&apikey=#{etherscan_apikey}"
playerbook_abi = JSON.parse(Faraday.get(playerbook_abi_url).body)["result"]

$playerbook_contract = Ethereum::Contract.create(
        client: $client, 
        name: playerbook_name, 
        address: playerbook_addr, 
        abi: playerbook_abi)

$playerbook_contract.key = sender_key
$playerbook_contract.gas_limit = 30_000_000
$playerbook_contract.gas_price = $client.eth_gas_price["result"].to_i(16)

nil

In [ ]:
decoder = Ethereum::Decoder.new

_abi_constructor, _abi_function, _abi_event = Ethereum::Abi.parse_abi($fomo_contract.abi)

event_abi = _abi_event.map do |x|
   ["0x"+x.signature,$fomo_contract.abi.filter {|y| y["name"]==x.name}[0]]
end.to_h


{"0x8f36579a548bc439baa172a6521207464154da77f411e2da3db2f53affe6cc3a"=>{"anonymous"=>false, "inputs"=>[{"indexed"=>true, "name"=>"playerID", "type"=>"uint256"}, {"indexed"=>false, "name"=>"playerAddress", "type"=>"address"}, {"indexed"=>false, "name"=>"playerName", "type"=>"bytes32"}, {"indexed"=>false, "name"=>"ethOut", "type"=>"uint256"}, {"indexed"=>false, "name"=>"timeStamp", "type"=>"uint256"}], "name"=>"onWithdraw", "type"=>"event"}, "0x00af92312107f6175aa18020d4c26cc83792028aa65c484d8c2833fe42584b59"=>{"anonymous"=>false, "inputs"=>[{"indexed"=>true, "name"=>"playerID", "type"=>"uint256"}, {"indexed"=>true, "name"=>"playerAddress", "type"=>"address"}, {"indexed"=>true, "name"=>"playerName", "type"=>"bytes32"}, {"indexed"=>false, "name"=>"roundID", "type"=>"uint256"}, {"indexed"=>false, "name"=>"ethIn", "type"=>"uint256"}, {"indexed"=>false, "name"=>"keys", "type"=>"uint256"}, {"indexed"=>false, "name"=>"timeStamp", "type"=>"uint256"}], "name"=>"onBuyKey", "type"=>"event"}, "0x38

In [ ]:
def get_balance(addr)
    $client.eth_get_balance(addr)["result"].to_i(16)/1e18.to_f
end

:get_balance

In [ ]:
# ===== View Account Balance =======

accounts = [sender_account,impersonate_account]

output = accounts.map do |addr|
  [addr, get_balance(addr)].join(" - ")
end.join("\n")

puts output

0xaCC961474eb17C103BBed96FC8f0E4eEac91c475 - 99.98950159685413
0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266 - 9899.980326880512


In [ ]:
## 操作账户转账到Sender account = 100 ETH
sender_balance = get_balance(sender_account)
impersonate_balance =  get_balance(impersonate_account)
amount = 100.to_i-sender_balance
$client.transfer(impersonate_key, sender_account, (amount*1e18).to_i) if amount>0


# ===== View Account Balance =======

accounts = [sender_account,impersonate_account]

output = accounts.map do |addr|
  [addr, get_balance(addr)].join(" - ")
end.join("\n")

puts output

0xaCC961474eb17C103BBed96FC8f0E4eEac91c475 - 99.99999999999999
0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266 - 9899.969595009115


In [ ]:
# # === Inspect PlayerBook ===


# puts "==Read=="
# puts $playerbook_contract.abi.filter{ |x| x["constant"]==true }
#     .filter{ |x| x["type"]=="function" }
#     .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

# puts "==Write=="
# puts $playerbook_contract.abi.filter{ |x| x["constant"]==false }
#     .filter{ |x| x["type"]=="function" }
#     .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

In [ ]:
# # 查询合约情况

# gID_ = $playerbook_contract.call.g_id_
# pID_ = $playerbook_contract.call.p_id_
# registrationFee_ = $playerbook_contract.call.registration_fee_
# getNameFee = $playerbook_contract.call.get_name_fee

# puts "gID_ = #{gID_}"
# puts "pID_ = #{pID_}"
# puts "registrationFee_ = #{registrationFee_}"
# puts "getNameFee = #{getNameFee}"

# addrs = Parallel.map( (1..pID_), in_threads: 20) do |i|
#     {pid:i,addr:$playerbook_contract.call.get_player_addr(i)}
# end

# games = (1..gID_).map do |i|
#     {gid:i, addr:$playerbook_contract.call.games_(i)}
# end


# puts
# puts "player addr:"
# puts addrs.map {|x| 
#     [x[:addr],
#      $playerbook_contract.call.get_player_name(x[:pid]),
#      $playerbook_contract.call.get_player_l_aff(x[:pid]) 
#      ]
#      .join(" - ") }.join("\n")
# puts "\n"
# puts "game addr:"
# puts games.map {|x| 
#     [x[:addr],
#      $playerbook_contract.call.game_names_(x[:addr]),
#      get_balance("0x"+x[:addr])]
#     .join(" - ")}.join("\n")


In [ ]:
# === Inspect Fomo ===


puts "==Read=="
puts $fomo_contract.abi.filter{ |x| x["constant"]==true }
    .filter{ |x| x["type"]=="function" }
    .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

puts "==Write=="
puts $fomo_contract.abi.filter{ |x| x["constant"]==false }
    .filter{ |x| x["type"]=="function" }
    .map {|x| [x["type"],x["name"],x["inputs"].to_s,x["outputs"].to_s].join(" - ") }.join("\n")

==Read==
function - pIDxAddr - [{"name"=>"", "type"=>"address"}] - [{"name"=>"", "type"=>"uint256"}]
function - getCards - [{"name"=>"_id", "type"=>"uint256"}] - [{"name"=>"", "type"=>"uint256"}, {"name"=>"", "type"=>"address"}, {"name"=>"", "type"=>"bytes32"}, {"name"=>"", "type"=>"uint256"}, {"name"=>"", "type"=>"uint256"}]
function - name - [] - [{"name"=>"", "type"=>"string"}]
function - getBuyPrice - [{"name"=>"_key", "type"=>"uint256"}] - [{"name"=>"", "type"=>"uint256"}]
function - potSplit - [{"name"=>"", "type"=>"uint256"}] - [{"name"=>"allPlayer", "type"=>"uint256"}, {"name"=>"lastWinner", "type"=>"uint256"}, {"name"=>"inviter1st", "type"=>"uint256"}, {"name"=>"inviter2nd", "type"=>"uint256"}, {"name"=>"inviter3rd", "type"=>"uint256"}, {"name"=>"key1st", "type"=>"uint256"}, {"name"=>"key2nd", "type"=>"uint256"}, {"name"=>"key3rd", "type"=>"uint256"}, {"name"=>"next", "type"=>"uint256"}]
function - getPlayerVaults - [] - [{"name"=>"", "type"=>"uint256"}, {"name"=>"", "type"=>"

In [ ]:
$fomo_contract.abi.filter{ |x| x["constant"]==true }
    .filter{ |x| x["type"]=="function" }
    .filter{ |x| x["inputs"]==[]}.map {|x| x["name"]}.map { |x| 
        puts "#{x} = #{$fomo_contract.call.send(x.underscore)}"
    }

puts

name = Bingo4Beast Long Official


IOError: Error: VM Exception while processing transaction: reverted with reason string 'eth less'

In [ ]:
puts $fomo_contract.call.p_id_player_round(1,0)

IOError: Error: VM Exception while processing transaction: reverted with reason string 'eth less'

In [ ]:
index = 1
players = []
loop do
    puts index
    player = $fomo_contract.call.plyr_(index)
    index=index+1 
    break if player[0]=="0000000000000000000000000000000000000000"
    players.push ({addr:player[0],pid:index-1})
end


1
2
3
4
5


In [ ]:
puts "player addr:"
total_vaults = 0
table = players.map {|x| 
# table = addrs.map {|x| 
     a=$fomo_contract.call.get_player_info_by_address(x[:addr])
     b=$fomo_contract.call.get_player_vaults(x[:pid])
     c=$fomo_contract.call.plyr_(x[:pid]) 

    total_vaults =total_vaults+ a[3]/1e18+a[4]/1e18+a[5]/1e18 #b[0]+b[1]+b[2]
    [x[:addr],a[1],a[2],a[3],a[4],a[5],a[6],b[0],b[1],b[2],c[5],c[6]]
}

header = %w(addr name keys  winnings  general  affiliate  round_eth winnings  general  affiliate lrnd laff)

html = """
  <table>
    <tr>
<%  header.each do |x| %>
        <th><%= x %></th>
<% end %>
    </tr>
    <%
       table.each_with_index do |x,i| 
       %>
    <tr>
        <td><%=x[0]%></td>
        <td><%=x[1]%></td>
        <td><%=x[2]/1e18%></td>
        <td><%=x[3]/1e18%></td>
        <td><%=x[4]/1e18%></td>
        <td><%=x[5]/1e18%></td>
        <td><%=x[6]/1e18%></td>
        <td><%=x[7]/1e18%></td>
        <td><%=x[8]/1e18%></td>
        <td><%=x[9]/1e18%></td>
        <td><%=x[10]%></td>
        <td><%=x[11]%></td>
    </tr>
    <% 
       end %>
  </table>
"""

IRuby.html ERB.new(html).result(binding)


player addr:


addr,name,keys,winnings,general,affiliate,round_eth,winnings,general,affiliate,lrnd,laff
a78cd12e5f2daf88023f0bfe119eac8b3f3dbc93,sunson,13.650971698084911,0.0,0.04692571997900529,0.0,0.01,0.0,0.06519860342115283,0.0,3,0
fb31eb7b96e413bebee61f5e3880938b937c2ef0,gogogo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
ea8a4f09c45967dfcfda180fa80ad44eefab52be,play,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
f1e32a3eaa5d6c360af6aa2c45a97e377be183bd,begin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [ ]:
puts "total_vaults - #{total_vaults}"
puts "contract_balance - #{get_balance(fomo_addr)}"
puts "diff = #{get_balance(fomo_addr)-total_vaults}"

total_vaults - 0.04692571997900529
contract_balance - 0.5598145300233387
diff = 0.5128888100443334


In [ ]:
rID_ = $fomo_contract.call.r_id_
round = (rID_-2..rID_).map do |i|
    $fomo_contract.call.round_(i)
end

round_team = (rID_-2..rID_).map do |i|
    [$fomo_contract.call.rnd_tm_eth_(i,0),$fomo_contract.call.rnd_tm_eth_(i,1),$fomo_contract.call.rnd_tm_eth_(i,2),$fomo_contract.call.rnd_tm_eth_(i,3)]
end


html = """
  <table>
    <tr>
<% %w(index plyr team team0 team1 team2 team3  end  ended strt duration keys eth keys_per_eth pot mask ico icoGen icoAvg).each do |x| %>
        <th><%= x %></th>
<% end %>
    </tr>
    <% acc_eth = 0
       round.each_with_index do |rnd,i| 
       %>
    <tr>
            <td> <%= i %> </td>
            <td> <%= rnd[0] %> </td>
            <td> <%= rnd[1] %> </td>
            <td> <%= round_team[i][0]/1e18 %> </td>
            <td> <%= round_team[i][1]/1e18 %> </td>
            <td> <%= round_team[i][2]/1e18 %> </td>
            <td> <%= round_team[i][3]/1e18 %> </td>
            <td> <%= Time.at(rnd[2]).to_fs(:db) %> </td>
            <td> <%= rnd[3] %> </td>
            <td> <%= Time.at(rnd[4]).to_fs(:db) %> </td>
            <td> <%= ActiveSupport::Duration.build(rnd[2]-rnd[4]).inspect %> </td>
            <td> <%= rnd[5]/1e18 %> </td>
            <td> <%= rnd[6]/1e18 %> </td>
            <td> <%= rnd[6]==0 ? 0 : rnd[5]/rnd[6] %> </td>
            <td> <%= rnd[7]/1e18 %> </td>
            <td> <%= rnd[8] %> </td>
            <td> <%= rnd[9] %> </td>
            <td> <%= rnd[10] %> </td>
            <td> <%= rnd[11] %> </td>
    </tr>
    <% 
       end %>
  </table>
"""

IRuby.html ERB.new(html).result(binding)

index,plyr,team,team0,team1,team2,team3,end,ended,strt,duration,keys,eth,keys_per_eth,pot,mask,ico,icoGen,icoAvg
0,1,2,0.0,0.0,0.000107625,0.0,2018-08-05 16:15:52,true,2018-08-05 13:15:52,3 hours,1.05,0.000107625,9756,1.722e-05,71750000000000,107625000000000,75337500000000,102500000000000
1,9,0,0.6415097169808491,0.01,0.0,0.0,2018-08-05 21:50:37,true,2018-08-05 18:08:07,"3 hours, 42 minutes, and 30 seconds",113.65097169808492,0.6515097169808491,174,0.16903403613200166,3950320986056289,10000000000000000,7000000000000000,732548584904245
2,187,3,0.0,0.0,0.0,0.0001,2022-03-19 18:36:07,false,2022-03-19 15:36:07,3 hours,0.0,0.0,0,1.6e-05,0,100000000000000,70000000000000,0


In [ ]:
player_round = (1..rID_).map do |r|
    player_round = (1..pID_).map do |p|
        $fomo_contract.call.plyr_rnds_(p,r)
    end
    puts "== round #{r} =="
    puts "# - eth - keys - mask - ico"
    puts player_round.map.with_index {|x,i|  [i,x[0]/1e18,x[1]/1e18,x[2]/1e18,x[3]/1e18,x[0]==0?".":x[2].to_f/x[0]].join(" - ") }.join("\n")
end

nil

NameError: undefined local variable or method `pID_' for #<Object:0x0000557a63f00ef0>

In [ ]:
## fees_ and potSplit_

# fees_[0] = F3Ddatasets.TeamFee(30,6);   //50% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot
# fees_[1] = F3Ddatasets.TeamFee(43,0);   //43% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot
# fees_[2] = F3Ddatasets.TeamFee(56,10);  //20% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot
# fees_[3] = F3Ddatasets.TeamFee(43,8);   //35% to pot, 10% to aff, 2% to com, 1% to pot swap, 1% to air drop pot

# potSplit_[0] = F3Ddatasets.PotSplit(15,10);  //48% to winner, 25% to next round, 2% to com
# potSplit_[1] = F3Ddatasets.PotSplit(25,0);   //48% to winner, 25% to next round, 2% to com
# potSplit_[2] = F3Ddatasets.PotSplit(20,20);  //48% to winner, 10% to next round, 2% to com
# potSplit_[3] = F3Ddatasets.PotSplit(30,10);  //48% to winner, 10% to next round, 2% to com



In [ ]:
code = '''
module Ethereum
  class Contract
    def send_raw_transaction(payload, to = nil)
      Eth.configure { |c| c.chain_id = @client.net_version["result"].to_i }
      @nonce ||= @client.get_nonce(key.address)
      args = {
        from: key.address,
        value: $transact_amount ? $transact_amount : 0,
        data: payload,
        nonce: @nonce,
        gas_limit: gas_limit,
        gas_price: gas_price
      }
      args[:to] = to if to
      tx = Eth::Tx.new(args)
      tx.sign key
      @client.eth_send_raw_transaction(tx.hex)["result"]
    end
  end
end
'''

def run_code(code,model)
  open("#{model}.rb","w") {|f| f.write(code) }
  load "./#{model}.rb"
end

run_code(code,"hotfix")


true

In [ ]:
### buyXid

$transact_amount = $fomo_contract.call.i_want_x_keys(1*1e18.to_i)
# $transact_amount = 750000000002000
team = 3

$fomo_contract.nonce = $client.get_nonce($fomo_contract.key.address)
tx = $fomo_contract.transact_and_wait.buy_xid(0,team)



#<Ethereum::Transaction:0x0000557a649c64c0 @mined=true, @connection=#<Ethereum::HttpClient:0x0000557a64c07180 @id=0, @log=false, @batch=nil, @formatter=#<Ethereum::Formatter:0x0000557a64c07158>, @gas_price=16005973946, @gas_limit=30000000, @host="127.0.0.1", @port=8545, @proxy=nil, @ssl=false, @uri=#<URI::HTTP http://127.0.0.1:8545/>, @default_account="0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266">, @id="0x0a97b96b8a268cd5bf69e31d64d27abac16e0b782149c5eca5b5fd10105dd8d8", @input="0x8f38f30900000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003", @input_parameters=[0, 3]>

In [ ]:
# tx_detail = $client.eth_get_transaction_by_hash(tx.id)["result"]
puts tx_detail.map {|x| x.join(" - ")}.join("\n")

puts "---------------"
puts "blockNumber - #{tx_detail["blockNumber"].to_i(16)}"
puts "value - #{tx_detail["value"].to_i(16)}"
puts "gas - #{tx_detail["gas"].to_i(16)}"
puts "gasPrice - #{tx_detail["gasPrice"].to_i(16)}"


blockHash - 0xf28567d14d525935f426fbdde7f72d56abca848bf3fa9228c4d8b91db75eb0e6
blockNumber - 0xdbfef3
from - 0xacc961474eb17c103bbed96fc8f0e4eeac91c475
gas - 0x1c9c380
hash - 0xcc779df4ccf351c615ac7f77dc3d858c07efdf202fcadb5cd8de0dbbcf269cb5
input - 0x3ccfd60b0000000000000000000000000000000000000000000000000000000000000000
nonce - 0x18
to - 0xf1ae594cefee0bf519f227f3262ee2a851b14b9a
transactionIndex - 0x0
value - 0x0
v - 0xf4f6
r - 0xf29d5376a8d2e8b266d38c6b8fb6033b1636a60cdca88198f78004d18fa9ddac
s - 0x1df368ee38efbb64f37549bb9751c1a552dd79444f158aec8a71f64d8bcc28e1
type - 0x0
gasPrice - 0x62691db37
---------------
blockNumber - 14417651
value - 0
gas - 30000000
gasPrice - 26416896823


In [ ]:
tx_receipt = $client.eth_get_transaction_receipt(tx.id)["result"]
puts tx_receipt.map {|x| x.join(" - ")}.join("\n")
puts "---------------"
puts "effectiveGasPrice - #{tx_receipt["effectiveGasPrice"].to_i(16)}"
puts "gasUsed - #{tx_receipt["gasUsed"].to_i(16)}"
puts "gas_fee - #{tx_receipt["gasUsed"].to_i(16) * tx_receipt["effectiveGasPrice"].to_i(16) / 1e18}"


transactionHash - 0x0a97b96b8a268cd5bf69e31d64d27abac16e0b782149c5eca5b5fd10105dd8d8
transactionIndex - 0x0
blockHash - 0x0301add7b88190fcb49ce2ac98b3522cb8b99f12ac684699c494f8bc37dd030d
blockNumber - 0xdbfef5
from - 0xacc961474eb17c103bbed96fc8f0e4eeac91c475
to - 0xb05d588c4852a9cb467cf9eded1f10130d48fad6
cumulativeGasUsed - 0x7e1b7
gasUsed - 0x7e1b7
contractAddress - 
logs - {"removed"=>false, "logIndex"=>"0x0", "transactionIndex"=>"0x0", "transactionHash"=>"0x0a97b96b8a268cd5bf69e31d64d27abac16e0b782149c5eca5b5fd10105dd8d8", "blockHash"=>"0x0301add7b88190fcb49ce2ac98b3522cb8b99f12ac684699c494f8bc37dd030d", "blockNumber"=>"0xdbfef5", "address"=>"0xb05d588c4852a9cb467cf9eded1f10130d48fad6", "data"=>"0x000000000000000000000000000000000000001d0d065b99b4f31210bfe8e5af000000000000000000006ae91c5255f4c03439524b0b42d52f48736a040000bb0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000acc961474eb17c103bbed96fc8f0e4eeac91c47500000000000000000000000000000000

In [ ]:
logs = tx_receipt["logs"][0]
data = logs["data"]
topic = logs["topics"]
puts logs.map {|x| x.join(" - ")}.join("\n")

removed - false
logIndex - 0x0
transactionIndex - 0x0
transactionHash - 0x0a97b96b8a268cd5bf69e31d64d27abac16e0b782149c5eca5b5fd10105dd8d8
blockHash - 0x0301add7b88190fcb49ce2ac98b3522cb8b99f12ac684699c494f8bc37dd030d
blockNumber - 0xdbfef5
address - 0xb05d588c4852a9cb467cf9eded1f10130d48fad6
data - 0x000000000000000000000000000000000000001d0d065b99b4f31210bfe8e5af000000000000000000006ae91c5255f4c03439524b0b42d52f48736a040000bb0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000acc961474eb17c103bbed96fc8f0e4eeac91c47500000000000000000000000000000000000000000000000000005af3107a400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000ce468fe1161cd1f86086e9bf5ee160ff322c7c96371637966313233000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000021c7a0116bd18670000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:

event={}
event["abi"] = event_abi[topic[0]]
event["name"] = event["abi"]["name"]
event_inputs = event["abi"]['inputs'].map {|i| OpenStruct.new(i)}
event["data"] = event_inputs.map {|x| x.name}.zip(decoder.decode_arguments(event_inputs, data)).to_h

puts "== Event: #{event["name"]} =="
puts event["data"].map{|x| [x[0],x[1]].join("-") }.join("\n")


== Event: onEndTx ==
compressedData-2301647704167001533505837000111
compressedIDs-40000000000000000000000000900000000000000000000000187
playerName-
playerAddress-acc961474eb17c103bbed96fc8f0e4eeac91c475
ethIn-100000000000000
keysBought-0
winnerAddr-0ce468fe1161cd1f86086e9bf5ee160ff322c7c9
winnerName-cqcyf123
amountWon-152130632518801511
newPot-0
P3DAmount-0
genAmount-70000000000000
potAmount-16000000000000
airDropPot-102076250000000


In [ ]:
puts "buy key amount - #{$transact_amount/1e18}"
puts "tx gas_fee - #{gas_fee = tx_receipt["gasUsed"].to_i(16) * tx_receipt["effectiveGasPrice"].to_i(16) / 1e18}"
puts "total_out - #{total_out = $transact_amount/1e18 + tx_receipt["gasUsed"].to_i(16) * tx_receipt["effectiveGasPrice"].to_i(16) / 1e18}"
puts "gas_fee percentage - #{gas_fee/total_out}"

puts "sender balance - #{get_balance(sender_account)}"


buy key amount - 0.0001
tx gas_fee - 0.00826764575219711
total_out - 0.00836764575219711
gas_fee percentage - 0.9880492072726976
sender balance - 99.9916323542478


In [ ]:

block_detail = $client.eth_get_block_by_number(tx_detail["blockNumber"].to_i(16),true)["result"]
puts block_detail.map {|x| x.join(" - ")}.join("\n")

puts "---------------"
puts "transactions.count - #{block_detail["transactions"].count}"


number - 0xdbfef3
hash - 0xf28567d14d525935f426fbdde7f72d56abca848bf3fa9228c4d8b91db75eb0e6
parentHash - 0xf5b120bc879cac8a31fc64f60e3eb094dc0f38fbb15d4d1203960122d1aa43e4
nonce - 0x0000000000000042
mixHash - 0x0000000000000000000000000000000000000000000000000000000000000000
sha3Uncles - 0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347
logsBloom - 0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000040000000000000010000000000000000001000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000200000000000000000000080000000000000000000000000000000000020000000040000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
transactionsRoot - 0x9daac6b8c81f195ec25248c13a756453fca257395732ca0578bba8641acc1d8c
stateRoot - 0xd2065345f5052e96c

In [ ]:
### withdraw
$transact_amount = 0
$fomo_contract.nonce = $client.get_nonce($fomo_contract.key.address)
tx = $fomo_contract.transact_and_wait.withdraw

#<Ethereum::Transaction:0x0000557a648465a0 @mined=true, @connection=#<Ethereum::HttpClient:0x0000557a64c07180 @id=0, @log=false, @batch=nil, @formatter=#<Ethereum::Formatter:0x0000557a64c07158>, @gas_price=16005973946, @gas_limit=30000000, @host="127.0.0.1", @port=8545, @proxy=nil, @ssl=false, @uri=#<URI::HTTP http://127.0.0.1:8545/>, @default_account="0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266">, @id="0xd9a91a71597bf3736632be015499fc306071f65698712a8aba7f0eb85589cdce", @input="0x3ccfd60b0000000000000000000000000000000000000000000000000000000000000000", @input_parameters=[]>

In [ ]:
tx_detail = $client.eth_get_transaction_by_hash(tx.id)["result"]
puts tx_detail.map {|x| x.join(" - ")}.join("\n")

puts "---------------"
puts "blockNumber - #{tx_detail["blockNumber"].to_i(16)}"
puts "value - #{tx_detail["value"].to_i(16)}"
puts "gas - #{tx_detail["gas"].to_i(16)}"
puts "gasPrice - #{tx_detail["gasPrice"].to_i(16)}"


blockHash - 0xf29208f4902977bfe9b598eaf4f4cc15c05b7a05703db3912e3fedc809b055dc
blockNumber - 0xdbfef6
from - 0xacc961474eb17c103bbed96fc8f0e4eeac91c475
gas - 0x1c9c380
hash - 0xd9a91a71597bf3736632be015499fc306071f65698712a8aba7f0eb85589cdce
input - 0x3ccfd60b0000000000000000000000000000000000000000000000000000000000000000
nonce - 0x1a
to - 0xb05d588c4852a9cb467cf9eded1f10130d48fad6
transactionIndex - 0x0
value - 0x0
v - 0xf4f6
r - 0xd5f912f7cc504ff811d6c4ef9ce87a622c9699a1f189269ecc185e28f7dd8a95
s - 0x35dd2d50f87caa35ba392abd736bab1d1c628ea4209cf3e466eb9d0be29b17b4
type - 0x0
gasPrice - 0x3ba07c7ba
---------------
blockNumber - 14417654
value - 0
gas - 30000000
gasPrice - 16005973946


In [ ]:
tx_receipt = $client.eth_get_transaction_receipt(tx.id)["result"]
puts tx_receipt.map {|x| x.join(" - ")}.join("\n")
puts "---------------"
puts "effectiveGasPrice - #{tx_receipt["effectiveGasPrice"].to_i(16)}"
puts "gasUsed - #{tx_receipt["gasUsed"].to_i(16)}"
puts "gas_fee - #{tx_receipt["gasUsed"].to_i(16) * tx_receipt["effectiveGasPrice"].to_i(16) / 1e18}"


transactionHash - 0xd9a91a71597bf3736632be015499fc306071f65698712a8aba7f0eb85589cdce
transactionIndex - 0x0
blockHash - 0xf29208f4902977bfe9b598eaf4f4cc15c05b7a05703db3912e3fedc809b055dc
blockNumber - 0xdbfef6
from - 0xacc961474eb17c103bbed96fc8f0e4eeac91c475
to - 0xb05d588c4852a9cb467cf9eded1f10130d48fad6
cumulativeGasUsed - 0x21918
gasUsed - 0x21918
contractAddress - 
logs - {"removed"=>false, "logIndex"=>"0x0", "transactionIndex"=>"0x0", "transactionHash"=>"0xd9a91a71597bf3736632be015499fc306071f65698712a8aba7f0eb85589cdce", "blockHash"=>"0xf29208f4902977bfe9b598eaf4f4cc15c05b7a05703db3912e3fedc809b055dc", "blockNumber"=>"0xdbfef6", "address"=>"0xb05d588c4852a9cb467cf9eded1f10130d48fad6", "data"=>"0x000000000000000000000000acc961474eb17c103bbed96fc8f0e4eeac91c475000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003faa25226000000000000000000000000000000000000000000000000000000000006235f89a", "topics"=>["0x8f36579a548bc43

In [ ]:
logs = tx_receipt["logs"][0]
data = logs["data"]
topic = logs["topics"]
puts "log count = #{tx_receipt["logs"].count}"
puts logs.map {|x| x.join(" - ")}.join("\n")

log count = 1
removed - false
logIndex - 0x0
transactionIndex - 0x0
transactionHash - 0xd9a91a71597bf3736632be015499fc306071f65698712a8aba7f0eb85589cdce
blockHash - 0xf29208f4902977bfe9b598eaf4f4cc15c05b7a05703db3912e3fedc809b055dc
blockNumber - 0xdbfef6
address - 0xb05d588c4852a9cb467cf9eded1f10130d48fad6
data - 0x000000000000000000000000acc961474eb17c103bbed96fc8f0e4eeac91c475000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003faa25226000000000000000000000000000000000000000000000000000000000006235f89a
topics - 0x8f36579a548bc439baa172a6521207464154da77f411e2da3db2f53affe6cc3a - 0x00000000000000000000000000000000000000000000000000000000000000bb


In [ ]:

event={}
event["abi"] = event_abi[topic[0]]
event["name"] = event["abi"]["name"]
event_inputs = event["abi"]['inputs'].map {|i| OpenStruct.new(i)}
event["data"] = event_inputs.map {|x| x.name}.zip(decoder.decode_arguments(event_inputs, data)).to_h

puts "== Event: #{event["name"]} =="
puts event["data"].map{|x| [x[0],x[1]].join("-") }.join("\n")


== Event: onWithdraw ==


JSON::GeneratorError: source sequence is illegal/malformed utf-8

In [ ]:
puts "tx gas_fee - #{gas_fee = tx_receipt["gasUsed"].to_i(16) * tx_receipt["effectiveGasPrice"].to_i(16) / 1e18}"
puts "sender balance - #{get_balance(sender_account)}"


tx gas_fee - 0.002200757393679216
sender balance - 99.98950159685413


In [ ]:
# buy key amount - 7.5e-05
# tx gas_fee - 0.012698015764806073
# total_out - 0.012773015764806073
# gas_fee percentage - 0.9941282465017658
# sender balance - 99.9872359842352

# tx gas_fee - 0.007280189317043226
# sender balance - 99.98104080726533

99.98104080726533-(99.9872359842352-0.007280189317043226)

0.0010850123471755069

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>